In [ ]:
# ----- From the 3rd place solution ----- #
from tqdm import tqdm
from openai import OpenAI
import os
import json
import dotenv
import pandas as pd
import re
import time
%pip install google-generativeai
import google.generativeai as genai

dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


In [ ]:
def generate_output(system_prompt, user_prompt):
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

    model = genai.GenerativeModel(
        model_name="gemini-2.5-flash",   # or gemini-1.5-pro
        system_instruction=system_prompt
    )

    response = model.generate_content(
        user_prompt,
        generation_config={
            "temperature": 0
        }
    )

    return response.text



global call_count
call_count = 0

# MultiHiertt
def replace_multi_table(text):
    call_count = 0
    instruction = "Retrieve all facts in the following table as natural language descriptions, without introductory phrases."
    
    table_pattern = r'(\|.*\|(?:\n\|.*\|)+)'
    tables = re.findall(table_pattern, text)
    
    for table in tables:
        replace_table = generate_output(instruction, table)
        text = text.replace(table, replace_table)
        
        call_count += 1
        if call_count % 1000 == 0: 
            time.sleep(60)  # Pause for 1 minute
        
    return text


def replace_table(text):
    global call_count
    instruction = "Retrieve all facts in the following table as natural language descriptions, without introductory phrases."

    table_lines = re.findall(r".*\|.*", text)
    table_text = "\n".join(table_lines)
    
    replace_text = generate_output(instruction, table_text)
    text = text.replace(table_text, replace_text)
    
    call_count += 1
    if call_count % 1000 == 0: 
        time.sleep(60)  # Pause for 1 minute
        
    return text

In [ ]:
tqdm.pandas()

tabular_retrieval = ['TATQA', 'FinQA', 'ConvFinQA', 'MultiHiertt']
                     
for task in tabular_retrieval:
    data = pd.read_csv(f"./dataset_csv/{task}_corpus.csv")
    data = data.dropna(subset=["text"]).reset_index(drop=True)
    
    if task == 'MultiHiertt':
        data['convert_text'] = data['text'].progress_apply(replace_multi_table)
    else:
        data['convert_text'] = data['text'].progress_apply(replace_table)
    
    #Save
    data.to_csv(f"./data/{task}_corpus_convert.csv", index=False)